## 1. mac_addr csv 가져오기

In [0]:
# csv path
csv_path = "file:/Workspace/EIC_데이터엔지니어/databricks_jira/eic/1218/mac_addr.csv"

# spark dataframe 
df = (spark.read
      .format("csv")
      .option("header", True)           # 첫 행을 컬럼명으로 사용
      .option("inferSchema", True)      # 간단한 경우 자동 스키마 추론
      # .option("delimiter", ",")       # 구분자 기본은 ',' (필요 시 지정)
      # .option("encoding", "utf-8")    # 인코딩 필요 시 지정
      .load(csv_path)
     )
    
# 맥 값 전처리 
from pyspark.sql import functions as F
df = df.withColumn(
    "mac_addr_2",
    F.concat_ws(
        ":",
        F.substring("MAC_ADDRESS", 1, 2),
        F.substring("MAC_ADDRESS", 3, 2),
        F.substring("MAC_ADDRESS", 5, 2),
        F.substring("MAC_ADDRESS", 7, 2),
        F.substring("MAC_ADDRESS", 9, 2),
        F.substring("MAC_ADDRESS", 11, 2),
    )
)

# 맥 해쉬 처리
tv_salt = dbutils.secrets.get('admin', 'salt')

df = df.withColumn("mac_addr_hashed", 
                   F.when(
                       F.col("mac_addr_2").isNull() | (F.col("mac_addr_2") == ''), 
                       None)\
                    .otherwise(F.sha2(F.concat(F.col("mac_addr_2"), F.lit(tv_salt)), 256)))

display(df)


In [0]:
import pandas as pd

# sdf -> pdf 
pdf = df.toPandas()
pdf.columns = ['_'.join(col.split(' ')) for col in pdf.columns]
pdf = pdf[['Production_Date'
           , 'Model.Suffix'
           , 'SET_ID'
           , 'MAC_ADDRESS'
           , 'mac_addr_hashed']]
pdf = pdf.reset_index(drop=False)
display(pdf)

In [0]:
spark.createDataFrame(pdf[['mac_addr_hashed']]).createOrReplaceTempView("tmp_mac_addr")

## 2. activation_date - min(crt_date) 구하기

In [0]:
df_result_1 = spark.sql(f''' 
    
    select raw.mac_addr
           , min(raw.crt_date) as min_crt_date -- 제공해야함
           , first(raw.Platform_code) as platform_code -- 제공해야함
           , max(raw.last_chg_date) as max_last_chg_date
           , date_format(max(raw.last_chg_date), 'yyyy-MM') as date_ym
    from   eic_data_ods.tlamp.activation_date raw
    inner join tmp_mac_addr tmp on raw.mac_addr = tmp.mac_addr_hashed
    group by mac_addr
''')
display(df_result_1)

## 3. 노말로그 값 구하기
> 3-1) max_last_chg_date를 yyyy-MM 로 바꾸고, date_ym별 mac_addr 대한 마지막 normal_log 추출 (union 활용) <br/>
> 3-2) 단, max_last_chg_date < '2025-12' 이면, 데이터 없음으로 추출 (2년 보관 중)

In [0]:
%sql 
SELECT
    X_device_platform as `platform_code`,  -- 필수 열
    X_device_product as `platform_version`,-- 필수 열
    X_Device_Country as `country_code`,    -- 필수/조건 열
    log_create_time,                       -- 필수/조건 열
    X_Device_SDK_VERSION as `dpv`,
    mac_addr as `device_id`,               -- 필수 열  
    x_device_sales_model as `sales_model_code`, -- 필수 열
    context_name,                          -- 필수/조건 열  
    message_id,                            -- 필수/조건 열
    normal_log
FROM eic_data_ods.tlamp.normal_log_webos23
WHERE 1=1
    AND X_device_country = 'TR' -- 튀르키예
    AND context_name = ''
    AND message_id = ''
    AND date_ym >= ''
    AND date_ym < ''
ORDER BY date_format(log_create_time, 'yyyy-MM')
 